# MQTT

MQTT and MQTT-SN are IP protocols used to manage devices in the internet of things.  These protocols implement a publish and subscribe communication protocol between *clients*.  The publish and subscribe communication is managed by *brokers*.  One of the key ideas of MQTT and MQTT-SN is that brokers are relatively robust and available, while clients are often transient (for instance they might often sleep to save power).  Typically a client will publish *events* to a broker.  Each event consists of a *topic* and a *message*.   Typically the broker will relay each event to other clients who have subscribed to the same topic, although brokers may also communicate directly to relay events.  The main difference between MQTT and MQTT-SN is that MQTT is based on TCP, while MQTT-SN is based on UDP.  For more information, see http://mqtt.org/documentation.

Paho is a readily available MQTT client.  We can use it to publish and subscribe to events.

In [ ]:
# Connect the client to our local server
import paho.mqtt.client as mqtt
client=mqtt.Client()
client.connect("localhost")


In the Paho API, subscribing to an event is done by registering a callback function.  We can use the callback to take an action, such as flashing the LEDs on the Pynq board.

In [ ]:
# Notice that messages aren't actually processed until one of the paho event loop functions is called
# e.g. loop() or loop_forever()
from pynq.board import LED

leds = [LED(index) for index in range(4)]

return_message=""
def on_message(client, userdata, message):
    print("Received message '" + str(message.payload) + "' on topic '" + message.topic + "' with QoS " + str(message.qos))
    for led in leds:
        led.toggle()    
    
client.on_message=on_message

client.subscribe("button/0")

Publishing an event is somewhat simpler, we simply need to provide the topic and the message

In [ ]:
(result, foo) = client.publish("button/0", 7)
result

In [ ]:
# Continually read the current state of the buttons and publish some corresponding MQTT messages.
# In addition, regularly call the 'loop()' function to process MQTT messages that
# are subscribed to.  Buttons are read until button 3 is pressed.
from pynq.board import Button

btns = [Button(index) for index in range(4)]
last = [btns[index].read() for index in range(3)]

while (btns[3].read()==0):
    current = [btns[index].read() for index in range(3)]
    changed = [current[index] != last[index] for index in range(3)]
    push = [client.publish('button/'+str(index),str(current[index])) if(changed[index]) else None for index in range(3)]
    last = current
    client.loop(timeout=.001)
        
print(current)
push = [client.publish('button/'+str(index),"offline") for index in range(3)]


Note that publishing events with MQTT can be quite fast, if we don't guarauntee that the events are received by the server.

In [ ]:
import timeit
start_time = timeit.default_timer()
# code you want to evaluate
for index in range(10000):
    client.publish("foo","bar")
elapsed = timeit.default_timer() - start_time
print(str(elapsed)+" seconds.")
print(str(10000/elapsed)+" messages/second.")